In [141]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    roc_auc_score, confusion_matrix, classification_report, f1_score
)

import warnings
warnings.filterwarnings("ignore", category=UserWarning)
pd.options.display.max_rows = None
pd.set_option('future.no_silent_downcasting', True)

In [2]:
import numpy as np
print(np.__version__)

1.26.4


In [212]:
ndf = pd.read_csv("../Downloads/st_exportd.csv").iloc[:,1:]
for i in range(len(ndf)):
    ndf.iloc[i,-3]=ndf.iloc[i,-3].split("GMT")[0].strip()
ndf['timestamp'] = pd.to_datetime(ndf['timestamp'])
a=np.random.randint(len(ndf))
ndf.iloc[a:a+5,:]

,klineacc,spread,spreadper,x,vwap,deviation,ratio,term,sigma,e,h,asset,timestamp,gap,gaplimit
10593,26561406,6.347698e+05,24.232344,0.833414,5.179189,0.020430,0.757677,0.348794,0.017934,0.301051,0.292274,ICP,2025-11-04 21:13:46,5m,60
10594,49509080,-5.238540e+06,-120.289795,-0.752206,2.266795,0.001767,2.202898,0.259299,0.003714,0.271695,0.265201,XRP,2025-11-04 21:13:46,5m,60
10595,19245970,-9.000852e+05,-47.339378,-0.804420,2.045270,0.000895,1.473394,0.279303,0.006083,0.060534,0.060460,SUI,2025-11-04 21:13:46,5m,60
10596,57053672,3.996601e+06,42.243893,0.770961,952.341003,-0.005713,0.577561,0.181282,0.004298,-0.139192,-0.138300,BNB,2025-11-04 21:13:46,5m,60
10597,352511168,4.503882e+06,36.823338,0.856473,104043.570312,-0.001878,0.631767,0.395834,0.002757,-0.170315,-0.168687,BTC,2025-11-04 21:13:46,5m,60


In [272]:
train_df=ndf.iloc[:110254,:]
actual_x=ndf.iloc[110254:,:]

In [273]:
assets_to_train=[i for i in actual_x["asset"].unique().tolist() if i in train_df["asset"].unique().tolist()]

In [250]:
c=[[len(train_df[train_df["asset"]==i]),i] for i in assets_to_train]

In [251]:
c = sorted(c, key=lambda x: x[0],reverse=True)
print(c)

[[10202, 'BTC'], [10182, 'ETH'], [9608, 'SOL'], [9597, 'ZEC'], [9521, 'BNB'], [9472, 'XRP'], [4852, 'SUI'], [3692, 'ICP'], [3306, 'DASH'], [3204, 'GIGGLE'], [3055, 'LTC'], [2954, 'LINK'], [2887, 'ZEN'], [2449, 'UNI'], [2403, 'NEAR'], [2345, 'TAO'], [2146, 'FIL'], [1918, 'TRUMP'], [1760, 'AVAX'], [1301, 'PAXG'], [1212, 'BCH'], [796, 'AAVE'], [661, 'AR'], [567, 'DOT'], [480, 'ETC'], [359, 'CAKE'], [359, 'RENDER'], [348, 'ORDI'], [344, 'WBTC'], [326, 'INJ'], [311, 'APT'], [288, 'TON'], [288, 'PENDLE'], [265, 'ATOM'], [250, 'TRB'], [204, 'NMR'], [194, 'WBETH'], [191, 'QNT'], [185, 'ZRO'], [125, 'ORCA'], [82, 'COMP'], [28, 'EGLD']]


In [256]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from datetime import timedelta
import joblib

# ---- user-visible outputs
OUT_DIR = Path("./eval_outputs")
MODELS_DIR = OUT_DIR / "asset_models"
PRED_CSV = OUT_DIR / "predictions_nndf_with_metrics.csv"
PER_ASSET_CSV = OUT_DIR / "per_asset_metrics.csv"
PER_ASSET_GAP_CSV = OUT_DIR / "per_asset_gap_metrics.csv"

RANDOM_STATE = 42
N_ESTIMATORS = 200

# ---- gap parsing (fixed)
def parse_gap_to_minutes(g):
    if pd.isna(g):
        return np.nan
    s = str(g).strip()
    # case-sensitive is fine; but treat 'M' same as 'm' only if user produced that
    if s.endswith("m"):
        return int(s[:-1])
    if s.endswith("h"):
        return int(s[:-1]) * 60
    # fallback: try numeric
    try:
        return int(s)
    except:
        return np.nan

# ---- fallback total windows (user-provided mapping)
GAP_TOTAL_MIN = {
    '1m': 300,
    '3m': 300,
    '5m': 300,
    '15m': 300,
    '30m': 300,
    '1h': 300,
    '2h': 7200,
    '4h': 7200,
    '6h': 7200,
    '8h': 7200,
    '12h': 7200
}

def ensure_dirs():
    OUT_DIR.mkdir(parents=True, exist_ok=True)
    MODELS_DIR.mkdir(parents=True, exist_ok=True)

df_raw = train_df.copy()
nndf = actual_x.copy()

# ensure timestamps
df_raw['timestamp'] = pd.to_datetime(df_raw['timestamp'])
nndf['timestamp'] = pd.to_datetime(nndf['timestamp'])

# compute gap_min in both
df_raw['gap_min'] = df_raw['gap'].apply(parse_gap_to_minutes)
nndf['gap_min'] = nndf['gap'].apply(parse_gap_to_minutes)

# --- compute true_next_vwap inside train (so we train on real next sample within same asset,gap)
df_sorted = df_raw.sort_values(['asset','gap','timestamp']).copy()
df_sorted['true_next_vwap'] = df_sorted.groupby(['asset','gap'])['vwap'].shift(-1)
train_rows = df_sorted.dropna(subset=['true_next_vwap']).copy()
print("Train rows with valid true_next_vwap:", len(train_rows))

# --- choose features: numeric columns except meta
meta_cols = {'asset','timestamp','gap','gaplimit','gap_min','true_next_vwap'}
numeric_cols = train_rows.select_dtypes(include=[np.number]).columns.tolist()
features = [c for c in numeric_cols if c not in meta_cols]
# ensure gap_min and gaplimit present as features (model can use them)
if 'gap_min' not in features:
    features.append('gap_min')
if 'gaplimit' not in features and 'gaplimit' in df_sorted.columns:
    features.append('gaplimit')
features = list(dict.fromkeys(features))
print("Using features:", features)

# --- train one model PER ASSET (all gaps included in training)
ensure_dirs()
assets = sorted(list(set(nndf['asset'].unique()) & set(df_raw['asset'].unique())))
print("Assets to train/evaluate:", assets)

asset_models = {}
train_stats = []

for asset in assets:
    rows = train_rows[train_rows['asset'] == asset]
    if len(rows) < 2:
        # not enough samples to train
        print(f"Skipping {asset} (insufficient training rows: {len(rows)})")
        continue

    X = rows[features].fillna(0).values
    y = rows['true_next_vwap'].values

    X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.15, random_state=RANDOM_STATE)
    model = RandomForestRegressor(n_estimators=N_ESTIMATORS, random_state=RANDOM_STATE, n_jobs=-1)
    model.fit(X_tr, y_tr)

    y_val_pred = model.predict(X_val)
    rmse_val = np.sqrt(np.mean((y_val - y_val_pred)**2))
    mae_val = mean_absolute_error(y_val, y_val_pred)
    r2 = r2_score(y_val, y_val_pred)

    model_path = MODELS_DIR / f"rf_{asset}.joblib"
    joblib.dump({'model': model, 'features': features}, model_path)

    asset_models[asset] = {'model': model, 'features': features, 'path': str(model_path)}

    train_stats.append({
        'asset': asset,
        'train_rows': len(rows),
        'val_RMSE': float(rmse_val),
        'val_MAE': float(mae_val),
        'val_R2': float(r2)
    })

pd.DataFrame(train_stats).to_csv(OUT_DIR/"per_asset_train_metrics.csv", index=False)
print("Saved per-asset train metrics.")

# --- produce one prediction per (asset,gap) using last row in df_raw for that gap
pred_records = []
gaps_universe = sorted(df_raw['gap'].dropna().unique().tolist(), key=str)  # up to 11 gaps

for asset, minfo in asset_models.items():
    model = minfo['model']
    feats = minfo['features']
    adf = df_raw[df_raw['asset'] == asset]

    for gap in gaps_universe:
        sub = adf[adf['gap'] == gap]
        if sub.empty:
            continue
        last = sub.sort_values('timestamp').iloc[-1]
        X_row = last[feats].fillna(0).values.reshape(1,-1)
        pred = float(model.predict(X_row)[0])

        pred_records.append({
            'asset': asset,
            'gap': gap,
            'gap_min': parse_gap_to_minutes(gap),
            'gaplimit': last.get('gaplimit', np.nan),
            'last_df_timestamp': last['timestamp'],
            'last_df_vwap': float(last['vwap']),
            'pred_next_vwap': pred,
            'model_path': minfo['path']
        })

pred_df = pd.DataFrame(pred_records)
print("Predictions produced (asset x gap):", len(pred_df))

# --- EVALUATION: use nndf == actual_x (rows after training end)
# window_minutes = gap_min * gaplimit if gaplimit exists, else fallback to GAP_TOTAL_MIN[gap]
eval_rows = []
for _, prow in pred_df.iterrows():
    asset = prow['asset']
    gap = prow['gap']
    gap_min = prow['gap_min']
    gaplimit = prow['gaplimit']
    last_ts = prow['last_df_timestamp']
    pred_val = prow['pred_next_vwap']
    last_vwap = prow['last_df_vwap']

    # select candidate future rows for same asset & gap
    cand = nndf[
        (nndf['asset'] == asset) &
        (nndf['gap'] == gap) &
        (nndf['timestamp'] > last_ts)
    ].copy()

    # decide horizon:
    if (pd.notna(gap_min) and pd.notna(gaplimit)):
        try:
            window_minutes = int(gap_min) * int(gaplimit)
        except:
            window_minutes = GAP_TOTAL_MIN.get(gap, 300)
    else:
        window_minutes = GAP_TOTAL_MIN.get(gap, 300)

    window_end = last_ts + timedelta(minutes=window_minutes)

    cand_sel = cand[cand['timestamp'] <= window_end]

    if cand_sel.empty:
        eval_rows.append({
            'asset': asset, 'gap': gap, 'count_truths': 0,
            'pred_next_vwap': pred_val, 'rmse': np.nan, 'mae': np.nan, 'direction_acc': np.nan,
            'window_end': window_end
        })
        continue

    truths = cand_sel['vwap'].astype(float).values
    errors = truths - pred_val
    rmse = float(np.sqrt(np.mean(errors**2)))
    mae = float(np.mean(np.abs(errors)))
    pred_dir = np.sign(pred_val - last_vwap)
    true_dirs = np.sign(truths - last_vwap)
    dir_acc = float((true_dirs == pred_dir).mean())

    eval_rows.append({
        'asset': asset,
        'gap': gap,
        'count_truths': int(len(truths)),
        'pred_next_vwap': float(pred_val),
        'rmse': rmse,
        'mae': mae,
        'direction_acc': dir_acc,
        'window_end': window_end
    })

eval_df = pd.DataFrame(eval_rows)

# --- per-asset aggregation (weighted by count_truths)
agg = []
for asset, g in eval_df.groupby('asset'):
    valid = g.dropna(subset=['rmse', 'mae', 'direction_acc'])
    if valid.empty:
        agg.append({'asset': asset, 'RMSE': np.nan, 'MAE': np.nan, 'Direction_Acc': np.nan,
                    'Count': int(g['count_truths'].sum())})
        continue
    weights = valid['count_truths'].values.astype(float)
    agg.append({
        'asset': asset,
        'RMSE': float(np.average(valid['rmse'], weights=weights)),
        'MAE': float(np.average(valid['mae'], weights=weights)),
        'Direction_Acc': float(np.average(valid['direction_acc'], weights=weights)),
        'Count': int(g['count_truths'].sum())
    })

per_asset_metrics_df = pd.DataFrame(agg).sort_values('RMSE', ascending=False)

# --- save outputs
ensure_dirs()
pred_full = pred_df.merge(eval_df, on=['asset','gap'], how='left')
pred_full.to_csv(PRED_CSV, index=False)
per_asset_metrics_df.to_csv(PER_ASSET_CSV, index=False)
eval_df.to_csv(PER_ASSET_GAP_CSV, index=False)

print("Saved outputs to", OUT_DIR)
print("Pipeline complete.")

Train rows with valid true_next_vwap: 109651
Using features: ['klineacc', 'spread', 'spreadper', 'x', 'vwap', 'deviation', 'ratio', 'term', 'sigma', 'e', 'h', 'gap_min', 'gaplimit']
Assets to train/evaluate: ['AAVE', 'APT', 'AR', 'ATOM', 'AVAX', 'BCH', 'BNB', 'BTC', 'CAKE', 'COMP', 'DASH', 'DOT', 'EGLD', 'ETC', 'ETH', 'FIL', 'GIGGLE', 'ICP', 'INJ', 'LINK', 'LTC', 'NEAR', 'NMR', 'ORCA', 'ORDI', 'PAXG', 'PENDLE', 'QNT', 'RENDER', 'SOL', 'SUI', 'TAO', 'TON', 'TRB', 'TRUMP', 'UNI', 'WBETH', 'WBTC', 'XRP', 'ZEC', 'ZEN', 'ZRO']
Saved per-asset train metrics.
Predictions produced (asset x gap): 404
Saved outputs to eval_outputs
Pipeline complete.


In [270]:
# ------------------- EVALUATION -------------------
from datetime import timedelta
import numpy as np
import pandas as pd

eval_rows = []

for _, prow in pred_df.iterrows():
    asset = prow['asset']
    gap = prow['gap']
    last_ts = prow['last_df_timestamp']
    gap_min = prow['gap_min']
    gaplimit = prow['gaplimit']
    pred_val = prow['pred_next_vwap']
    last_vwap = prow['last_df_vwap']

    cand = nndf[(nndf['asset'] == asset) &
                (nndf['gap'] == gap) &
                (nndf['timestamp'] > last_ts)]
    GAP_TOTAL_MIN = {
        '1m': 300,
        '3m': 300,
        '5m': 300,
        '15m': 300,
        '30m': 300,
        '1h': 300,
        '2h': 7200,
        '4h': 7200,
        '6h': 7200,
        '8h': 7200,
        '12h': 7200,
    }
    total_min = GAP_TOTAL_MIN[gap]
    window_end = last_ts + timedelta(minutes=total_min)
    cand_sel = cand[cand['timestamp'] <= window_end]

    if cand_sel.empty:
        eval_rows.append({
            'asset': asset,
            'gap': gap,
            'count_truths': 0,
            'pred_next_vwap': pred_val,
            'rmse': np.nan,
            'mae': np.nan,
            'direction_acc': np.nan
        })
        continue

    truths = cand_sel['vwap'].values.astype(float)
    errors = truths - pred_val
    rmse = np.sqrt(np.mean(errors**2))
    mae = np.mean(np.abs(errors))

    # direction accuracy
    pred_dir = np.sign(pred_val - last_vwap)
    true_dirs = np.sign(truths - last_vwap)
    dir_acc = np.mean(true_dirs == pred_dir)

    eval_rows.append({
        'asset': asset,
        'gap': gap,
        'count_truths': len(truths),
        'pred_next_vwap': pred_val,
        'rmse': float(rmse),
        'mae': float(mae),
        'direction_acc': float(dir_acc)
    })

eval_df = pd.DataFrame(eval_rows)

# ------------------- AGGREGATE PER-ASSET -------------------
agg_records = []
for asset, g in eval_df.groupby('asset'):
    valid = g.dropna(subset=['rmse', 'mae', 'direction_acc'])
    if valid.empty:
        agg_records.append({
            'asset': asset,
            'RMSE': np.nan,
            'MAE': np.nan,
            'Direction_Acc': np.nan,
            'Count': int(g['count_truths'].sum())
        })
        continue
    counts = valid['count_truths'].values
    rmse_w = np.average(valid['rmse'], weights=counts)
    mae_w = np.average(valid['mae'], weights=counts)
    diracc_w = np.average(valid['direction_acc'], weights=counts)
    agg_records.append({
        'asset': asset,
        'RMSE': float(rmse_w),
        'MAE': float(mae_w),
        'Direction_Acc': float(diracc_w),
        'Count': int(g['count_truths'].sum())
    })

per_asset_metrics_df = pd.DataFrame(agg_records).sort_values('RMSE', ascending=False)

# ------------------- RESULTS -------------------
print("Per-asset-gap prediction evaluation:")
eval_df.dropna(subset=['rmse', 'mae', 'direction_acc'])
print("\nPer-asset aggregated metrics:")
per_asset_metrics_df.dropna(subset=['RMSE', 'MAE', 'Direction_Acc'])

Per-asset-gap prediction evaluation:


,asset,gap,count_truths,pred_next_vwap,rmse,mae,direction_acc
0,AAVE,12h,3,181.981115,8.273380,8.180913,0.000000
4,AAVE,2h,19,181.480123,8.267253,7.645294,0.000000
7,AAVE,4h,9,181.406924,8.072313,7.407707,1.000000
9,AAVE,6h,6,181.679175,8.041689,7.566281,1.000000
10,AAVE,8h,4,181.427874,8.070115,7.927961,0.000000
11,APT,12h,3,1.933146,0.125957,0.118510,0.000000
14,APT,2h,19,1.941917,0.116770,0.107406,0.000000
17,APT,4h,9,1.929826,0.100064,0.088920,1.000000
19,APT,6h,6,1.933299,0.110054,0.101857,0.000000
20,APT,8h,4,1.931785,0.105725,0.093290,1.000000



Per-asset aggregated metrics:


,asset,RMSE,MAE,Direction_Acc,Count
37,WBTC,1539.448915,1417.725879,0.951220,41
7,BTC,1003.824986,917.418709,0.793814,97
14,ETH,71.612794,65.643521,0.666667,96
5,BCH,20.116331,18.620531,0.463415,41
6,BNB,14.602781,13.658786,0.525773,97
16,GIGGLE,11.171876,10.975702,1.000000,41
25,PAXG,10.674571,10.001551,0.682927,41
39,ZEC,9.545856,7.424263,0.453608,97
0,AAVE,8.172667,7.648347,0.365854,41
31,TAO,5.802384,5.257776,0.219512,41
